# 🎉 AI Fact-Check & Fix-My-Code Assistant

1. A user enters a claim (e.g., “The earth is flat.”).
2. A user optionally adds Python code they want fixed (e.g. "def avg(lst):
    return sum(lst) / len(lst) + 10"). 

The app uses:

1. *web_search* tool to verify factual statements
2. *code_interpreter* to execute and correct Python code

The assistant returns:

1. A fact-checked explanation using search results
2. A corrected, working Python code (if code is provided)

Everything happens through a simple **Gradio UI**.

In [ ]:
import os
import gradio as gr
from openai import OpenAI
from IPython.display import Markdown, display
from dotenv import load_dotenv

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")


In [ ]:
client = OpenAI(
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1"
)

In [ ]:
# ------------------------------------------------
# Main assistant logic
# ------------------------------------------------
def run_assistant(claim, user_code):

    instructions = """
        You are an AI fact-checker and Python code-fixing assistant.

        You must do ALL of the following:
        1. FACT-CHECK the user's claim using the web_search tool.
        2. If Python code is provided:
        – Analyze it
        – Fix errors
        – Improve structure
        – Test it using the code_interpreter tool
        3. Use the custom tool "clean_text" to clean the claim text.
        4. Return:
        – Fact-check conclusion + citations
        – Cleaned claim
        – Corrected code
        – Code execution results
        """
    
    prompt = f"""
        Follow a clear structure:
        FACT CHECK:
        CLEANED CLAIM:
        FIXED CODE:
        EXECUTION RESULT:
        NOTES:

        USER CLAIM: 
        {claim}

        USER CODE:
        {user_code}
    """

    # Call the OpenAI Responses API
    response = client.responses.create(
        model="gpt-4o-mini",
        instructions=instructions,
        input=prompt,
        tools=[
            {
                "type": "web_search"
            },
            {
                "type": "code_interpreter",
                "container": {"type": "auto", "memory_limit": "4g"}
            },
        ]
    )
    return response.output_text


In [ ]:
result = run_assistant("The Moon is made of cheese", "def multiply(a,b): return a + b")

In [ ]:
display(Markdown(result))

In [ ]:
# ------------------------------------------------
# Gradio User Interface
# ------------------------------------------------
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 AI Fact-Check & Fix-My-Code Assistant")
    gr.Markdown(
        "Enter a *claim* (e.g., 'Sharks are mammals', 'The Moon is made of cheese', ...) and optionally enter buggy Python code.\n"
        "The AI will fact-check your claim, fix & test the Python code, and show results."
    )

    claim_box = gr.Textbox(label="Enter a claim to fact-check", lines=2)
    code_box = gr.Textbox(label="Optional Python code to fix", lines=8)

    run_btn = gr.Button("Run Assistant")

    output_box = gr.Markdown(label="Assistant Output")

    run_btn.click(run_assistant, inputs=[claim_box, code_box], outputs=output_box)

demo.launch()